<a href="https://colab.research.google.com/github/albercej/zug/blob/master/prophet_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pystan==2.19.1.1 --quiet
!pip install fbprophet --quiet
!pip install yfinance --quiet

     |████████████████████████████████| 6.3 MB 12.7 MB/s 


In [2]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

In [3]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2016-01-01'

eth_df = yf.download('ETH-USD',start_date, today)

eth_df.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-11-24,"$4,340.04","$4,376.88","$4,176.18","$4,239.98","$4,239.98",21838037128
2021-11-25,"$4,271.39","$4,550.52","$4,249.77","$4,274.74","$4,274.74",18705358318
2021-11-26,"$4,522.21","$4,550.84","$3,933.51","$4,030.91","$4,030.91",26281795488
2021-11-27,"$4,043.00","$4,187.68","$4,033.51","$4,096.91","$4,096.91",16515693874
2021-11-28,"$4,081.45","$4,127.38","$3,995.28","$4,083.95","$4,083.95",13435139072


In [4]:
eth_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2155 entries, 2016-01-01 to 2021-11-28
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2155 non-null   float64
 1   High       2155 non-null   float64
 2   Low        2155 non-null   float64
 3   Close      2155 non-null   float64
 4   Adj Close  2155 non-null   float64
 5   Volume     2155 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 117.9 KB


In [5]:
eth_df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [6]:
eth_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [7]:
eth_df.reset_index(inplace=True)
eth_df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [8]:
df = eth_df[["Date", "Open"]]

new_names = {
    "Date": "ds", 
    "Open": "y",
}

df.rename(columns=new_names, inplace=True)

In [9]:
df.tail()

,ds,y
2150,2021-11-24,"$4,340.04"
2151,2021-11-25,"$4,271.39"
2152,2021-11-26,"$4,522.21"
2153,2021-11-27,"$4,043.00"
2154,2021-11-28,"$4,081.45"


In [10]:
# plot the open price

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [11]:
m = Prophet(
    seasonality_mode="multiplicative" 
)

m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [12]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
2515,2022-11-24
2516,2022-11-25
2517,2022-11-26
2518,2022-11-27
2519,2022-11-28


In [13]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2515,2022-11-24,"$7,583.75","$6,845.66","$8,346.74"
2516,2022-11-25,"$7,484.12","$6,754.35","$8,248.41"
2517,2022-11-26,"$7,336.75","$6,595.48","$8,096.66"
2518,2022-11-27,"$7,233.04","$6,479.22","$7,964.20"
2519,2022-11-28,"$7,078.72","$6,378.86","$7,805.60"


In [14]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

3823.2539343628364

In [16]:
plot_plotly(m, forecast)

In [17]:
plot_components_plotly(m, forecast)